In [ ]:
#!pip install datasets
#!pip install tensordict
#!pip install torchrl
#!pip install torcheval

In [38]:
from transformers import MobileBertConfig, MobileBertModel
from torch.utils.data import DataLoader, Dataset
from torch.optim import AdamW
from sklearn.metrics import accuracy_score, classification_report
from torchvision.transforms import ToTensor
import torch
from torch import nn
from sklearn.model_selection import train_test_split
from collections import OrderedDict
from transformers import AutoTokenizer, MobileBertModel,  BertTokenizer, BertForSequenceClassification
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from torchrl.data import TensorDictTokenizer
from datasets import load_dataset
import datasets
import random
import pandas as pd
import numpy as np
from transformers import Trainer, TrainingArguments
from torcheval.metrics import BinaryAccuracy

In [39]:
#Размер батча
batch_size = 16

In [40]:
# Загрузка данных
data = pd.read_csv('imdb dataset.csv')  
data = data[['review', 'sentiment']]  # 
print(data.head())


                                              review sentiment
0  One of the other reviewers has mentioned that ...  positive
1  A wonderful little production. <br /><br />The...  positive
2  I thought this was a wonderful way to spend ti...  positive
3  Basically there's a family where a little boy ...  negative
4  Petter Mattei's "Love in the Time of Money" is...  positive


In [41]:
# Преобразование меток в числовой формат
data['label'] = data['sentiment'].map({'positive': 1, 'negative': 0})
print(data.head())

                                              review sentiment  label
0  One of the other reviewers has mentioned that ...  positive      1
1  A wonderful little production. <br /><br />The...  positive      1
2  I thought this was a wonderful way to spend ti...  positive      1
3  Basically there's a family where a little boy ...  negative      0
4  Petter Mattei's "Love in the Time of Money" is...  positive      1


In [42]:
# Разделение на обучающую и тестовую   и валидационную выборки
train_texts, test_texts, train_labels, test_labels = train_test_split(
    data['review'].tolist(),
    data['label'].tolist(),
    test_size=0.2,
    random_state=42
)
val_texts, test_texts, val_labels, test_labels = train_test_split(
    test_texts,
    test_labels,
    test_size=0.5,
    random_state=42
)
print(len(train_texts))
print(len(test_texts))
print(len(val_texts))

print(len(train_labels))
print(len(test_labels))
print(len(val_labels))

40000
5000
5000
40000
5000
5000


In [43]:
#Создание устройства, на котором будут выполняться задачи, преимущество графическому процессору.
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#model.to(device)
device

device(type='cuda')

In [44]:
#Создание кастомной модели из стандартной берт модели. Ее не смог использовать на обучении. Ошибка tuple out of index.
class CustomBERTModel(torch.nn.Module):
    def __init__(self, pretrained_model_name, num_labels):
        super(CustomBERTModel, self).__init__()
        self.bert = AutoModelForSequenceClassification.from_pretrained(pretrained_model_name, num_labels=num_labels)
        for param in self.bert.parameters():
            param.requires_grad = False
        # Keep only the classification head trainable
        for param in self.bert.classifier.parameters():
            param.requires_grad = True
        self.dropout = torch.nn.Dropout(0.3)
        self.fc = torch.nn.Linear(self.bert.config.hidden_size, num_labels)

    def forward(self, input_ids, attention_mask):
        output = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = self.dropout(output(1))  # Applying dropout
        logits = self.fc(pooled_output)  # Adding a fully connected layer
        return logits


#Создание обычной модели из берт .
model_usual = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)
# Замораживание весов общей модели.
for param in model_usual.bert.parameters():
    param.requires_grad = False
# Оставление размороженными только выходные слои классификатора.
for param in model_usual.classifier.parameters():
    param.requires_grad = True
model_usual.to(device)    


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [45]:
#Создание класса датасета для даталоадера.  В методе гетитем значения превращаются в тензоры, 
# т.к именно тензоры требует энкодер.
class IMDbDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)    
    

In [46]:
max_length=512  # Максимальная длина токена
# Инициализация токенизатора BERT

tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
# Токенизация данных
train_encodings = tokenizer(train_texts, return_tensors='pt', truncation=True, padding=True, max_length=max_length)
test_encodings = tokenizer(test_texts, return_tensors='pt',truncation=True, padding=True, max_length=max_length)
val_encodings = tokenizer(val_texts, return_tensors='pt',truncation=True, padding=True, max_length=max_length)


  0%|          | 0/2500 [20:31<?, ?it/s]


In [47]:
# Создание трех токенизированных датасетов. Проверка формата.
train_dataset = IMDbDataset(train_encodings, train_labels)
test_dataset = IMDbDataset(test_encodings, test_labels)
val_dataset = IMDbDataset(val_encodings, val_labels)

val_dataset[2]

C:\Users\Антон\AppData\Local\Temp\ipykernel_5976\3800549983.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


{'input_ids': tensor([  101,  2065,  2017,  2066,  2010,  2265,  2017,  2453,  2022,  1037,
          2210,  9364,  1012,  2023,  3185,  2038,  2070,  2200,  6057,  5312,
          1998,  1996, 11680,  2024,  3492,  5377,  2021,  3904,  2024,  2200,
         13432,  2030,  2004,  6057,  2004,  1996,  2477,  2006,  1996,  2265,
          1012,  1996,  2927,  5537,  2003,  2428,  2428, 10021,  1998,  6057,
          1010,  1998,  1037,  2307,  2707,  1012,  2748,   999,  8945,  8609,
          2515,  2191,  1037, 12081,  3311,  1012,  1026,  7987,  1013,  1028,
          1026,  7987,  1013,  1028,  2065,  2017,  2024,  1037,  5470,  2059,
          3422,  2009,   999,  2065,  2017,  2123,  1005,  1056,  2113,  2032,
          2030,  2123,  1005,  1056,  2066,  2032,  2059,  2123,  1005,  1056,
          8572,  1012,  1020,  1012,  1019,  1013,  2184,   102,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,   

In [48]:
batch_size=16
learning_rate = 0.0001
num_epochs = 3

train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

In [ ]:
# Обучение модели
from tqdm import tqdm
# Оптимизатор и функция потерь
criterion = torch.nn.CrossEntropyLoss()
#optimizer = AdamW(model.parameters(), lr=learning_rate)
optimizer = AdamW(model_usual.parameters(), lr=learning_rate)

# Функция обучения
def train_model(model, train_dataloader, num_epochs):
    model.train()
    metric = BinaryAccuracy(threshold=0.5)
    for epoch in range(num_epochs):
        loop = tqdm(train_dataloader, leave=True)
        total_loss = 0
        batch_number = 0
        for batch in train_dataloader:
            #print("batch number",batch_number)
            batch_number+=1
            input_ids, attention_mask, labels = batch["token_type_ids"].to(device), batch["attention_mask"].to(device), batch["labels"].to(device)

            optimizer.zero_grad()
            outputs = model(input_ids, attention_mask=attention_mask)
            loss = criterion(outputs.logits, labels)
            #metric.update(outputs, labels)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
            if batch_number%100==0:
                print("batch number",batch_number)
                #print("accuracy",metric)
                print("loss",loss)
        
        print(f"Epoch {epoch+1}, Loss: {total_loss/len(train_dataloader):.4f}")
        loop.set_description(f"Epoch {epoch}")
        loop.set_postfix(loss=loss.item())

# Обучение модели

train_model(model_usual, train_dataloader, num_epochs)
#train_model(model, train_dataloader, num_epochs)


  0%|          | 0/2500 [00:00<?, ?it/s]C:\Users\Антон\AppData\Local\Temp\ipykernel_5976\3800549983.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


batch number 100
loss tensor(0.7080, device='cuda:0', grad_fn=<NllLossBackward0>)
batch number 200
loss tensor(0.7062, device='cuda:0', grad_fn=<NllLossBackward0>)
batch number 300
loss tensor(0.6977, device='cuda:0', grad_fn=<NllLossBackward0>)
batch number 400
loss tensor(0.7392, device='cuda:0', grad_fn=<NllLossBackward0>)
batch number 500
loss tensor(0.7006, device='cuda:0', grad_fn=<NllLossBackward0>)
batch number 600
loss tensor(0.7098, device='cuda:0', grad_fn=<NllLossBackward0>)


KeyboardInterrupt: 